# Matplotlib Practice — SOLUTIONS
### Dataset: AusApparalSales4thQrt2020.csv (Australian Apparel Sales Q4 2020)

**⚠️ Try solving the questions yourself first before looking at solutions!**

Open `matplotlib_practice.ipynb` to attempt the questions, then come here to verify your answers.

In [ ]:
# Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

df = pd.read_csv('../AusApparalSales4thQrt2020.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.strip()
df['Month'] = df['Date'].dt.month_name()

print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"States: {df['State'].unique()}")
print(f"Groups: {df['Group'].unique()}")
df.head()

---
## Section 1: Basic Line Plots

**Q1.** Daily total sales line chart with title and labels.

In [ ]:
# Q1 Solution
daily_sales = df.groupby('Date')['Sales'].sum().sort_index()

plt.figure(figsize=(12, 5))
plt.plot(daily_sales.index, daily_sales.values)
plt.title('Daily Total Sales - Q4 2020')
plt.xlabel('Date')
plt.ylabel('Total Sales ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Q2.** Daily total sales for each State on the same chart with a legend.

In [ ]:
# Q2 Solution
plt.figure(figsize=(12, 5))
for state in df['State'].unique():
    state_data = df[df['State'] == state].groupby('Date')['Sales'].sum().sort_index()
    plt.plot(state_data.index, state_data.values, label=state)

plt.title('Daily Total Sales by State - Q4 2020')
plt.xlabel('Date')
plt.ylabel('Total Sales ($)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Q3.** Daily total sales for Kids group only — red dashed line with circle markers.

In [ ]:
# Q3 Solution
kids_daily = df[df['Group'] == 'Kids'].groupby('Date')['Sales'].sum().sort_index()

plt.figure(figsize=(12, 5))
plt.plot(kids_daily.index, kids_daily.values, 'r--o', markersize=4)
plt.title('Daily Sales - Kids Group')
plt.xlabel('Date')
plt.ylabel('Total Sales ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Q4.** Monthly total sales line chart with markers. Annotate the highest month.

In [ ]:
# Q4 Solution
monthly_sales = df.groupby(df['Date'].dt.month)['Sales'].sum()
month_labels = ['Oct', 'Nov', 'Dec']

plt.figure(figsize=(8, 5))
plt.plot(month_labels, monthly_sales.values, 'bo-', markersize=10)

max_idx = monthly_sales.values.argmax()
plt.annotate(f'Highest: ${monthly_sales.values[max_idx]:,.0f}',
             xy=(month_labels[max_idx], monthly_sales.values[max_idx]),
             xytext=(0, 20), textcoords='offset points',
             arrowprops=dict(arrowstyle='->', color='red'),
             fontsize=12, color='red')

plt.title('Monthly Total Sales - Q4 2020')
plt.xlabel('Month')
plt.ylabel('Total Sales ($)')
plt.tight_layout()
plt.show()

**Q5.** 7-day rolling average overlaid on actual daily sales.

In [ ]:
# Q5 Solution
daily_sales = df.groupby('Date')['Sales'].sum().sort_index()
rolling_avg = daily_sales.rolling(window=7).mean()

plt.figure(figsize=(12, 5))
plt.plot(daily_sales.index, daily_sales.values, alpha=0.4, label='Daily Sales')
plt.plot(rolling_avg.index, rolling_avg.values, color='red', linewidth=2, label='7-Day Rolling Avg')
plt.title('Daily Sales with 7-Day Rolling Average')
plt.xlabel('Date')
plt.ylabel('Sales ($)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

---
## Section 2: Bar Charts

**Q6.** Vertical bar chart of total sales per State with colors and value labels.

In [ ]:
# Q6 Solution
state_sales = df.groupby('State')['Sales'].sum()
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

plt.figure(figsize=(8, 5))
bars = plt.bar(state_sales.index, state_sales.values, color=colors[:len(state_sales)])

for bar in bars:
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
             f'${bar.get_height():,.0f}', ha='center', va='bottom', fontsize=10)

plt.title('Total Sales by State')
plt.xlabel('State')
plt.ylabel('Total Sales ($)')
plt.tight_layout()
plt.show()

**Q7.** Horizontal bar chart of total sales per Group, sorted by value.

In [ ]:
# Q7 Solution
group_sales = df.groupby('Group')['Sales'].sum().sort_values()

plt.figure(figsize=(8, 5))
plt.barh(group_sales.index, group_sales.values, color=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99'])
plt.title('Total Sales by Group')
plt.xlabel('Total Sales ($)')
plt.ylabel('Group')
plt.tight_layout()
plt.show()

**Q8.** Grouped bar chart comparing total sales per State for each Time period.

In [ ]:
# Q8 Solution
pivot = df.pivot_table(values='Sales', index='State', columns='Time', aggfunc='sum')

x = np.arange(len(pivot.index))
width = 0.25

plt.figure(figsize=(10, 6))
for i, time in enumerate(pivot.columns):
    plt.bar(x + i * width, pivot[time], width, label=time)

plt.xticks(x + width, pivot.index)
plt.title('Total Sales by State and Time Period')
plt.xlabel('State')
plt.ylabel('Total Sales ($)')
plt.legend()
plt.tight_layout()
plt.show()

**Q9.** Stacked bar chart of total sales per State, stacked by Group.

In [ ]:
# Q9 Solution
pivot = df.pivot_table(values='Sales', index='State', columns='Group', aggfunc='sum')

plt.figure(figsize=(8, 6))
pivot.plot(kind='bar', stacked=True, ax=plt.gca())
plt.title('Total Sales by State (Stacked by Group)')
plt.xlabel('State')
plt.ylabel('Total Sales ($)')
plt.legend(title='Group')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

**Q10.** Grouped bar chart of average units sold per Group for each month.

In [ ]:
# Q10 Solution
pivot = df.pivot_table(values='Unit', index='Group', columns='Month', aggfunc='mean')

x = np.arange(len(pivot.index))
width = 0.25
months = ['October', 'November', 'December']

plt.figure(figsize=(10, 6))
for i, month in enumerate(months):
    if month in pivot.columns:
        plt.bar(x + i * width, pivot[month], width, label=month)

plt.xticks(x + width, pivot.index)
plt.title('Average Units Sold per Group by Month')
plt.xlabel('Group')
plt.ylabel('Average Units')
plt.legend()
plt.tight_layout()
plt.show()

---
## Section 3: Scatter Plots

**Q11.** Scatter plot of Unit vs Sales.

In [ ]:
# Q11 Solution
plt.figure(figsize=(8, 6))
plt.scatter(df['Unit'], df['Sales'], alpha=0.3, s=10)
plt.title('Unit vs Sales')
plt.xlabel('Unit')
plt.ylabel('Sales ($)')
plt.tight_layout()
plt.show()

**Q12.** Scatter plot of Unit vs Sales, colored by Group.

In [ ]:
# Q12 Solution
colors_map = {'Kids': 'red', 'Men': 'blue', 'Women': 'green', 'Seniors': 'orange'}

plt.figure(figsize=(8, 6))
for group, color in colors_map.items():
    mask = df['Group'] == group
    plt.scatter(df.loc[mask, 'Unit'], df.loc[mask, 'Sales'],
                alpha=0.3, s=10, color=color, label=group)

plt.title('Unit vs Sales by Group')
plt.xlabel('Unit')
plt.ylabel('Sales ($)')
plt.legend()
plt.tight_layout()
plt.show()

**Q13.** Scatter plot of Unit vs Sales with size representing Sales value.

In [ ]:
# Q13 Solution
plt.figure(figsize=(8, 6))
plt.scatter(df['Unit'], df['Sales'], s=df['Sales']/500, alpha=0.5, c='steelblue')
plt.title('Unit vs Sales (size = Sales)')
plt.xlabel('Unit')
plt.ylabel('Sales ($)')
plt.tight_layout()
plt.show()

**Q14.** Scatter plot of average daily Unit vs average daily Sales per state, labeled.

In [ ]:
# Q14 Solution
state_avg = df.groupby('State')[['Unit', 'Sales']].mean()

plt.figure(figsize=(8, 6))
plt.scatter(state_avg['Unit'], state_avg['Sales'], s=100, c='coral')

for state in state_avg.index:
    plt.annotate(state, (state_avg.loc[state, 'Unit'], state_avg.loc[state, 'Sales']),
                 textcoords='offset points', xytext=(10, 5), fontsize=12)

plt.title('Average Unit vs Average Sales per State')
plt.xlabel('Average Unit')
plt.ylabel('Average Sales ($)')
plt.tight_layout()
plt.show()

**Q15.** Scatter plot of Unit vs Sales colored by Time period.

In [ ]:
# Q15 Solution
time_colors = {'Morning': 'gold', 'Afternoon': 'skyblue', 'Evening': 'darkblue'}

plt.figure(figsize=(8, 6))
for time, color in time_colors.items():
    mask = df['Time'] == time
    plt.scatter(df.loc[mask, 'Unit'], df.loc[mask, 'Sales'],
                alpha=0.3, s=10, color=color, label=time)

plt.title('Unit vs Sales by Time Period')
plt.xlabel('Unit')
plt.ylabel('Sales ($)')
plt.legend()
plt.tight_layout()
plt.show()

---
## Section 4: Histograms

**Q16.** Histogram of Sales with 30 bins and a vertical line at the mean.

In [ ]:
# Q16 Solution
plt.figure(figsize=(8, 5))
plt.hist(df['Sales'], bins=30, edgecolor='black', alpha=0.7)
plt.axvline(df['Sales'].mean(), color='red', linestyle='--', linewidth=2, label=f"Mean: ${df['Sales'].mean():,.0f}")
plt.title('Distribution of Sales')
plt.xlabel('Sales ($)')
plt.ylabel('Frequency')
plt.legend()
plt.tight_layout()
plt.show()

**Q17.** Overlapping histograms of Sales for Morning, Afternoon, and Evening.

In [ ]:
# Q17 Solution
plt.figure(figsize=(8, 5))
for time in ['Morning', 'Afternoon', 'Evening']:
    plt.hist(df[df['Time'] == time]['Sales'], bins=30, alpha=0.5, label=time, edgecolor='black')

plt.title('Sales Distribution by Time Period')
plt.xlabel('Sales ($)')
plt.ylabel('Frequency')
plt.legend()
plt.tight_layout()
plt.show()

**Q18.** Histogram of Unit values with 20 bins and density=True.

In [ ]:
# Q18 Solution
plt.figure(figsize=(8, 5))
plt.hist(df['Unit'], bins=20, density=True, edgecolor='black', alpha=0.7, color='teal')
plt.title('Unit Distribution (Probability Density)')
plt.xlabel('Unit')
plt.ylabel('Density')
plt.tight_layout()
plt.show()

**Q19.** Side-by-side histograms of Sales for Kids and Seniors using subplots.

In [ ]:
# Q19 Solution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

ax1.hist(df[df['Group'] == 'Kids']['Sales'], bins=25, edgecolor='black', color='skyblue')
ax1.set_title('Kids Sales')
ax1.set_xlabel('Sales ($)')
ax1.set_ylabel('Frequency')

ax2.hist(df[df['Group'] == 'Seniors']['Sales'], bins=25, edgecolor='black', color='salmon')
ax2.set_title('Seniors Sales')
ax2.set_xlabel('Sales ($)')

plt.tight_layout()
plt.show()

**Q20.** Histogram of Sales for each State in a 2x2 subplot grid.

In [ ]:
# Q20 Solution
states = df['State'].unique()[:4]
fig, axes = plt.subplots(2, 2, figsize=(12, 8), sharey=True)

for ax, state in zip(axes.flatten(), states):
    ax.hist(df[df['State'] == state]['Sales'], bins=25, edgecolor='black', alpha=0.7)
    ax.set_title(f'{state} Sales Distribution')
    ax.set_xlabel('Sales ($)')
    ax.set_ylabel('Frequency')

plt.tight_layout()
plt.show()

---
## Section 5: Pie Charts

**Q21.** Pie chart of total sales by State with percentage labels.

In [ ]:
# Q21 Solution
state_sales = df.groupby('State')['Sales'].sum()

plt.figure(figsize=(8, 8))
plt.pie(state_sales, labels=state_sales.index, autopct='%1.1f%%', startangle=90)
plt.title('Sales Distribution by State')
plt.tight_layout()
plt.show()

**Q22.** Pie chart of sales by Group, exploding the largest slice.

In [ ]:
# Q22 Solution
group_sales = df.groupby('Group')['Sales'].sum()
explode = [0.1 if v == group_sales.max() else 0 for v in group_sales.values]

plt.figure(figsize=(8, 8))
plt.pie(group_sales, labels=group_sales.index, autopct='%1.1f%%',
        explode=explode, startangle=90, shadow=True)
plt.title('Sales Distribution by Group')
plt.tight_layout()
plt.show()

**Q23.** Pie chart of sales by Time period.

In [ ]:
# Q23 Solution
time_sales = df.groupby('Time')['Sales'].sum()

plt.figure(figsize=(8, 8))
plt.pie(time_sales, labels=time_sales.index, autopct='%1.1f%%', startangle=90,
        colors=['#FFD700', '#87CEEB', '#191970'])
plt.title('Sales Distribution by Time Period')
plt.tight_layout()
plt.show()

**Q24.** Donut chart for sales by State.

In [ ]:
# Q24 Solution
state_sales = df.groupby('State')['Sales'].sum()

plt.figure(figsize=(8, 8))
wedges, texts, autotexts = plt.pie(state_sales, labels=state_sales.index,
                                    autopct='%1.1f%%', startangle=90, pctdistance=0.85)

centre_circle = plt.Circle((0, 0), 0.70, fc='white')
plt.gca().add_artist(centre_circle)
plt.title('Sales by State (Donut Chart)')
plt.tight_layout()
plt.show()

---
## Section 6: Box Plots

**Q25.** Box plot of Sales for each State.

In [ ]:
# Q25 Solution
states = df['State'].unique()
data = [df[df['State'] == s]['Sales'].values for s in states]

plt.figure(figsize=(8, 5))
plt.boxplot(data, labels=states)
plt.title('Sales Distribution by State')
plt.xlabel('State')
plt.ylabel('Sales ($)')
plt.tight_layout()
plt.show()

**Q26.** Box plot of Sales for each Group with filled colors.

In [ ]:
# Q26 Solution
groups = df['Group'].unique()
data = [df[df['Group'] == g]['Sales'].values for g in groups]
colors = ['#FF9999', '#66B2FF', '#99FF99', '#FFCC99']

plt.figure(figsize=(8, 5))
bp = plt.boxplot(data, labels=groups, patch_artist=True)

for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

plt.title('Sales Distribution by Group')
plt.xlabel('Group')
plt.ylabel('Sales ($)')
plt.tight_layout()
plt.show()

**Q27.** Box plot of Unit for each Time period with horizontal line at overall median.

In [ ]:
# Q27 Solution
times = df['Time'].unique()
data = [df[df['Time'] == t]['Unit'].values for t in times]

plt.figure(figsize=(8, 5))
plt.boxplot(data, labels=times)
plt.axhline(y=df['Unit'].median(), color='red', linestyle='--', label=f"Overall Median: {df['Unit'].median()}")
plt.title('Unit Distribution by Time Period')
plt.xlabel('Time')
plt.ylabel('Unit')
plt.legend()
plt.tight_layout()
plt.show()

**Q28.** Side-by-side box plots: Sales by State and Units by State.

In [ ]:
# Q28 Solution
states = df['State'].unique()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

data_sales = [df[df['State'] == s]['Sales'].values for s in states]
ax1.boxplot(data_sales, labels=states)
ax1.set_title('Sales by State')
ax1.set_ylabel('Sales ($)')

data_units = [df[df['State'] == s]['Unit'].values for s in states]
ax2.boxplot(data_units, labels=states)
ax2.set_title('Units by State')
ax2.set_ylabel('Units')

plt.tight_layout()
plt.show()

---
## Section 7: Subplots & Multi-Panel Figures

**Q29.** 2x2 subplot: line plot, bar chart, histogram, pie chart.

In [ ]:
# Q29 Solution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Top-left: Line plot
daily = df.groupby('Date')['Sales'].sum().sort_index()
axes[0, 0].plot(daily.index, daily.values)
axes[0, 0].set_title('Daily Total Sales')
axes[0, 0].tick_params(axis='x', rotation=45)

# Top-right: Bar chart
state_sales = df.groupby('State')['Sales'].sum()
axes[0, 1].bar(state_sales.index, state_sales.values, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
axes[0, 1].set_title('Total Sales per State')

# Bottom-left: Histogram
axes[1, 0].hist(df['Sales'], bins=30, edgecolor='black', alpha=0.7)
axes[1, 0].set_title('Sales Distribution')

# Bottom-right: Pie chart
group_sales = df.groupby('Group')['Sales'].sum()
axes[1, 1].pie(group_sales, labels=group_sales.index, autopct='%1.1f%%')
axes[1, 1].set_title('Sales by Group')

plt.tight_layout()
plt.show()

**Q30.** 1x3 subplot: bar charts of total sales per Group for each Time period.

In [ ]:
# Q30 Solution
fig, axes = plt.subplots(1, 3, figsize=(16, 5), sharey=True)

for ax, time in zip(axes, ['Morning', 'Afternoon', 'Evening']):
    data = df[df['Time'] == time].groupby('Group')['Sales'].sum()
    ax.bar(data.index, data.values)
    ax.set_title(f'{time}')
    ax.set_xlabel('Group')
    ax.tick_params(axis='x', rotation=45)

axes[0].set_ylabel('Total Sales ($)')
plt.suptitle('Sales per Group by Time Period', fontsize=14)
plt.tight_layout()
plt.show()

**Q31.** 2x2 subplot with histograms of Sales for each state, shared y-axis.

In [ ]:
# Q31 Solution
states = df['State'].unique()[:4]
fig, axes = plt.subplots(2, 2, figsize=(12, 8), sharey=True)

for ax, state in zip(axes.flatten(), states):
    ax.hist(df[df['State'] == state]['Sales'], bins=25, edgecolor='black', alpha=0.7)
    ax.set_title(f'{state}')
    ax.set_xlabel('Sales ($)')

axes[0, 0].set_ylabel('Frequency')
axes[1, 0].set_ylabel('Frequency')
plt.suptitle('Sales Distribution by State', fontsize=14)
plt.tight_layout()
plt.show()

**Q32.** 3 subplots stacked vertically: daily sales for Oct, Nov, Dec.

In [ ]:
# Q32 Solution
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=False)
months = [(10, 'October'), (11, 'November'), (12, 'December')]

for ax, (month_num, month_name) in zip(axes, months):
    month_data = df[df['Date'].dt.month == month_num].groupby('Date')['Sales'].sum().sort_index()
    ax.plot(month_data.index, month_data.values)
    ax.set_title(f'{month_name} Daily Sales')
    ax.set_ylabel('Sales ($)')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

---
## Section 8: Customization & Styling

**Q33.** Customized bar chart with ggplot style, grid, removed spines, custom colors and fonts.

In [ ]:
# Q33 Solution
plt.style.use('ggplot')

state_sales = df.groupby('State')['Sales'].sum()
colors = ['#E74C3C', '#3498DB', '#2ECC71', '#F39C12']

fig, ax = plt.subplots(figsize=(8, 5))
bars = ax.bar(state_sales.index, state_sales.values, color=colors[:len(state_sales)])

ax.yaxis.grid(True, alpha=0.3)
ax.xaxis.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax.set_title('Total Sales by State', fontsize=16, fontweight='bold')
ax.set_xlabel('State', fontsize=12)
ax.set_ylabel('Total Sales ($)', fontsize=12)
ax.tick_params(labelsize=10)

for bar in bars:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
            f'${bar.get_height():,.0f}', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()
plt.style.use('default')  # Reset style

**Q34.** Line plot of WA daily sales with fill_between and peak annotation.

In [ ]:
# Q34 Solution
wa_daily = df[df['State'] == 'WA'].groupby('Date')['Sales'].sum().sort_index()

plt.figure(figsize=(12, 5))
plt.plot(wa_daily.index, wa_daily.values, color='#E74C3C', linewidth=2)
plt.fill_between(wa_daily.index, wa_daily.values, alpha=0.2, color='#E74C3C')

peak_date = wa_daily.idxmax()
peak_val = wa_daily.max()
plt.annotate(f'Peak: ${peak_val:,.0f}\n{peak_date.strftime("%b %d")}',
             xy=(peak_date, peak_val),
             xytext=(30, 20), textcoords='offset points',
             arrowprops=dict(arrowstyle='->', color='black'),
             fontsize=11, fontweight='bold')

plt.title('WA Daily Sales - Q4 2020')
plt.xlabel('Date')
plt.ylabel('Sales ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Q35.** Professional bar chart with error bars, average line, and rotated labels.

In [ ]:
# Q35 Solution
state_mean = df.groupby('State')['Sales'].mean()
state_std = df.groupby('State')['Sales'].std()
overall_avg = df['Sales'].mean()

plt.figure(figsize=(8, 5))
plt.bar(state_mean.index, state_mean.values, yerr=state_std.values,
        capsize=5, color='steelblue', alpha=0.8)
plt.axhline(y=overall_avg, color='red', linestyle='--', linewidth=2,
            label=f'Overall Avg: ${overall_avg:,.0f}')

plt.title('Average Sales by State (with Std Dev)', fontsize=14)
plt.xlabel('State', fontsize=12)
plt.ylabel('Average Sales ($)', fontsize=12)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

**Q36.** Save a plot as PNG (300 dpi), PDF, and SVG.

In [ ]:
# Q36 Solution
fig, ax = plt.subplots(figsize=(8, 5))
state_sales = df.groupby('State')['Sales'].sum()
ax.bar(state_sales.index, state_sales.values, color='steelblue')
ax.set_title('Total Sales by State')

fig.savefig('sales_chart.png', dpi=300, bbox_inches='tight')
fig.savefig('sales_chart.pdf', bbox_inches='tight')
fig.savefig('sales_chart.svg', bbox_inches='tight')

print("Saved: sales_chart.png, sales_chart.pdf, sales_chart.svg")
plt.show()

---
## Section 9: Heatmaps & Advanced Plots

**Q37.** Heatmap of average sales: State (rows) vs Time (columns) with annotations.

In [ ]:
# Q37 Solution
pivot = df.pivot_table(values='Sales', index='State', columns='Time', aggfunc='mean')

fig, ax = plt.subplots(figsize=(8, 5))
im = ax.imshow(pivot.values, cmap='YlOrRd', aspect='auto')

ax.set_xticks(range(len(pivot.columns)))
ax.set_yticks(range(len(pivot.index)))
ax.set_xticklabels(pivot.columns)
ax.set_yticklabels(pivot.index)

for i in range(len(pivot.index)):
    for j in range(len(pivot.columns)):
        ax.text(j, i, f'${pivot.values[i, j]:,.0f}', ha='center', va='center', fontsize=11)

plt.colorbar(im)
plt.title('Average Sales: State vs Time')
plt.tight_layout()
plt.show()

**Q38.** Heatmap of average sales: Group (rows) vs Month (columns).

In [ ]:
# Q38 Solution
pivot = df.pivot_table(values='Sales', index='Group', columns='Month', aggfunc='mean')

fig, ax = plt.subplots(figsize=(8, 5))
im = ax.imshow(pivot.values, cmap='Blues', aspect='auto')

ax.set_xticks(range(len(pivot.columns)))
ax.set_yticks(range(len(pivot.index)))
ax.set_xticklabels(pivot.columns)
ax.set_yticklabels(pivot.index)

for i in range(len(pivot.index)):
    for j in range(len(pivot.columns)):
        ax.text(j, i, f'${pivot.values[i, j]:,.0f}', ha='center', va='center', fontsize=11)

plt.colorbar(im)
plt.title('Average Sales: Group vs Month')
plt.tight_layout()
plt.show()

**Q39.** Stacked area chart of daily total sales by Group.

In [ ]:
# Q39 Solution
pivot = df.pivot_table(values='Sales', index='Date', columns='Group', aggfunc='sum').sort_index()

plt.figure(figsize=(12, 6))
plt.stackplot(pivot.index, [pivot[col] for col in pivot.columns],
              labels=pivot.columns, alpha=0.8)
plt.title('Daily Sales by Group (Stacked Area)')
plt.xlabel('Date')
plt.ylabel('Sales ($)')
plt.legend(loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

**Q40.** Twin y-axes: total Sales (left) and total Units (right) vs Date.

In [ ]:
# Q40 Solution
daily = df.groupby('Date').agg({'Sales': 'sum', 'Unit': 'sum'}).sort_index()

fig, ax1 = plt.subplots(figsize=(12, 5))

ax1.plot(daily.index, daily['Sales'], color='blue', label='Sales')
ax1.set_xlabel('Date')
ax1.set_ylabel('Total Sales ($)', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax1.tick_params(axis='x', rotation=45)

ax2 = ax1.twinx()
ax2.plot(daily.index, daily['Unit'], color='red', label='Units')
ax2.set_ylabel('Total Units', color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title('Daily Sales and Units')
fig.legend(loc='upper left', bbox_to_anchor=(0.12, 0.88))
plt.tight_layout()
plt.show()

---
## Section 10: Real-World Data Analysis Visualizations

**Q41.** Sales Dashboard: 4 subplots with suptitle.

In [ ]:
# Q41 Solution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Q4 2020 Australian Apparel Sales Dashboard', fontsize=16, fontweight='bold')

# 1. Sales trend
daily = df.groupby('Date')['Sales'].sum().sort_index()
axes[0, 0].plot(daily.index, daily.values, color='steelblue')
axes[0, 0].set_title('Sales Trend Over Time')
axes[0, 0].tick_params(axis='x', rotation=45)

# 2. Sales by state
state_sales = df.groupby('State')['Sales'].sum().sort_values(ascending=False)
axes[0, 1].bar(state_sales.index, state_sales.values, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'])
axes[0, 1].set_title('Sales Breakdown by State')

# 3. Sales distribution
axes[1, 0].hist(df['Sales'], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
axes[1, 0].axvline(df['Sales'].mean(), color='red', linestyle='--')
axes[1, 0].set_title('Sales Distribution')

# 4. Top group
group_sales = df.groupby('Group')['Sales'].sum().sort_values()
axes[1, 1].barh(group_sales.index, group_sales.values, color='coral')
axes[1, 1].set_title('Top Group Performance')

plt.tight_layout()
plt.show()

**Q42.** Daily sales for each state in separate subplots with shared axes.

In [ ]:
# Q42 Solution
states = df['State'].unique()
n = len(states)
cols = 2
rows = (n + 1) // 2

fig, axes = plt.subplots(rows, cols, figsize=(14, 4 * rows), sharex=True, sharey=True)

for ax, state in zip(axes.flatten(), states):
    state_daily = df[df['State'] == state].groupby('Date')['Sales'].sum().sort_index()
    ax.plot(state_daily.index, state_daily.values)
    ax.set_title(state)
    ax.tick_params(axis='x', rotation=45)

# Hide unused subplots
for ax in axes.flatten()[n:]:
    ax.set_visible(False)

plt.suptitle('Daily Sales by State', fontsize=14)
plt.tight_layout()
plt.show()

**Q43.** Bar chart of top 10 dates by total sales, #1 highlighted.

In [ ]:
# Q43 Solution
daily = df.groupby('Date')['Sales'].sum().sort_values(ascending=False).head(10)
colors = ['gold' if i == 0 else 'steelblue' for i in range(len(daily))]

plt.figure(figsize=(12, 6))
plt.bar(range(len(daily)), daily.values, color=colors)
plt.xticks(range(len(daily)), [d.strftime('%b %d') for d in daily.index], rotation=45)
plt.title('Top 10 Dates by Total Sales')
plt.xlabel('Date')
plt.ylabel('Total Sales ($)')
plt.tight_layout()
plt.show()

**Q44.** Grouped bar chart: Morning vs Evening sales per state with percentage difference.

In [ ]:
# Q44 Solution
me_data = df[df['Time'].isin(['Morning', 'Evening'])]
pivot = me_data.pivot_table(values='Sales', index='State', columns='Time', aggfunc='sum')

x = np.arange(len(pivot.index))
width = 0.35

plt.figure(figsize=(10, 6))
plt.bar(x - width/2, pivot['Morning'], width, label='Morning', color='gold')
plt.bar(x + width/2, pivot['Evening'], width, label='Evening', color='darkblue')

# Add percentage difference labels
for i, state in enumerate(pivot.index):
    pct_diff = ((pivot.loc[state, 'Evening'] - pivot.loc[state, 'Morning']) / pivot.loc[state, 'Morning']) * 100
    max_val = max(pivot.loc[state, 'Morning'], pivot.loc[state, 'Evening'])
    plt.text(i, max_val + 5000, f'{pct_diff:+.1f}%', ha='center', fontsize=10, fontweight='bold')

plt.xticks(x, pivot.index)
plt.title('Morning vs Evening Sales by State')
plt.xlabel('State')
plt.ylabel('Total Sales ($)')
plt.legend()
plt.tight_layout()
plt.show()

**Q45.** Monthly comparison chart: 3 bars (Oct, Nov, Dec) per state.

In [ ]:
# Q45 Solution
pivot = df.pivot_table(values='Sales', index='State', columns=df['Date'].dt.month_name(), aggfunc='sum')
months = ['October', 'November', 'December']
pivot = pivot[months]

x = np.arange(len(pivot.index))
width = 0.25
colors = ['#2ECC71', '#3498DB', '#E74C3C']

plt.figure(figsize=(10, 6))
for i, (month, color) in enumerate(zip(months, colors)):
    plt.bar(x + i * width, pivot[month], width, label=month, color=color)

plt.xticks(x + width, pivot.index)
plt.title('Monthly Sales Comparison by State', fontsize=14)
plt.xlabel('State')
plt.ylabel('Total Sales ($)')
plt.legend()
plt.tight_layout()
plt.show()

---
## ✅ All 45 Matplotlib Solutions Complete!

Go back to `matplotlib_practice.ipynb` and try any questions you couldn't solve.